# Mouse TTN遺伝子上のmotif配列の統計および可視化

### mouse_motif_comb.ipynbの内容
複数個の塩基を組み合わせた配列を作成し、それらの各イントロン領域における出現頻度を解析

### 準備
```
gbk = Seq_count() # クラスのインスタンス化
gbk.read_gbk('data/gbk/human_ttn.gb')　# gbkファイルの読み込み
```

### 特定のイントロン領域の組み合わせ配列の統計をデータフレームで返す
イントロン88のの配列の統計情報
組み合わせ配列の塩基数:4
```
gbk.intron_motif_stats(intron=88, n=4)
```

### 指定したインロンの配列を返す
```
gbk.intron_seq(77) # 77番目のイントロン配列を取得
```


## 準備

In [ ]:
# オリジナルモジュールのインポート
from lib.introngap import PileUp
from lib.gbkparse import Seq_count

# モジュールのインポート
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# クラスのインスタンス化
gbk = Seq_count()

# gbkファイルの読み込み
gbk.read_gbk('../data/gbk/mouse_ttn.gb')

In [ ]:
# 登録されているmRNAのIDを取得
for i in gbk.get_mrna_ids():
    gbk.set_mrna_id(i)
    print(i,gbk.intron_num())

In [ ]:

# トランスクリプトバリアントの設定
gbk.set_mrna_id('NM_001385708.1')

## n個の塩基の組み合わせ配列を作成

In [ ]:
# n個の塩基の組み合わせ配列を作成
# intron_motif_stats関数で、すべてイントロン領域の配列の個数をカウント
# その後、カウントした個数を全体の個数で割ることで、頻度を算出
n = 4
df = gbk.intron_motif_stats(intron=1, n=n) 
df.columns = ['intron_1']
for i in range(gbk.intron_num()-1):
    tmp_df = gbk.intron_motif_stats(intron=i+2, n=n)
    tmp_df.columns = [f'intron_{i+2}']
    df = pd.merge(df, tmp_df, left_index=True, right_index=True)
ndf = df/df.sum(axis=0)

## 組み合わせ配列の出現頻度をもとにもとに主成分分析を行う

In [ ]:
# データの標準化
scaler = StandardScaler()
ndf_scaled = scaler.fit_transform(ndf)

# PCAの実行
pca = PCA(n_components=2)
components = pca.fit_transform(ndf_scaled)

# PCAの結果をデータフレームに変換
pca_ndf = pd.DataFrame(data=components, columns=['PC1', 'PC2'])

# 元の特徴量をPCAの結果に結合
pca_ndf = pd.concat([pca_ndf, ndf.reset_index()], axis=1)

# Plotly Expressを使用した2D散布図の作成
# マウスオーバー時に表示する情報を追加
fig = px.scatter(
    pca_ndf, x='PC1', y='PC2', hover_name='index', 
    title="sequence motif PCA",
)
fig.show()

In [ ]:
# データの転置
tndf = ndf.T

# データの標準化
scaler = StandardScaler()
tndf_scaled = scaler.fit_transform(tndf)

# PCAの実行
pca = PCA(n_components=2)
components = pca.fit_transform(tndf_scaled)

# PCAの結果をデータフレームに変換
pca_tndf = pd.DataFrame(data=components, columns=['PC1', 'PC2'])

# 元の特徴量をPCAの結果に結合
pca_tndf = pd.concat([pca_tndf, tndf.reset_index()], axis=1)

# Plotly Expressを使用した2D散布図の作成
# マウスオーバー時に表示する情報を追加
fig = px.scatter(
    pca_tndf, x='PC1', y='PC2', hover_name='index', 
    title="Intron Motif PCA"
)
fig.show()

## 因子負荷量の計算

In [ ]:
# 因子負荷量の計算
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

# 結果の表示
loadings_df = pd.DataFrame(loadings, columns=['PC1', 'PC2'], index=tndf.columns)
ldf = pd.DataFrame(loadings_df)
ldf